In [1]:
from PIL import Image, ImageFont, ImageDraw, ImageChops
import os
import math

def get_dynamic_font(dpi, height_in_inches=0.25, font_name="arial.ttf"):
    from matplotlib import font_manager
    font_path = font_manager.findfont(font_name)
    font_size_px = int(height_in_inches * dpi)
    try:
        font = ImageFont.truetype(font_path, font_size_px)
    except Exception as e:
        print("Failed to load truetype font:", e)
        font = ImageFont.load_default()
    return font


def resize_to_match_height(img, target_height):
    w, h = img.size
    new_width = int(w * target_height / h)
    return img.resize((new_width, target_height), Image.LANCZOS)

def combine_images(image_paths, output_path, columns=2, background_color='white', margin=0, index_h=0.25, resize=False):
    labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)']
    if not image_paths:
        raise ValueError("No images provided.")

    # Open all images
    images = []
    for i, p in enumerate(image_paths):
        img = Image.open(p).convert('RGB')
        print(img.size)

        # Create a background image
        bg = Image.new('RGB', img.size, background_color)

        # Get the difference between image and background
        diff = ImageChops.difference(img, bg)

        # Get the bounding box of the non-background area
        bbox = diff.getbbox()
        if bbox is None:
            print("No figure content detected.")
            return img  # return original

        # Apply margin
        left = max(bbox[0] - margin, 0)
        upper = max(bbox[1] - margin, 0)
        right = min(bbox[2] + margin, img.width)
        lower = min(bbox[3] + margin, img.height)

        cropped_img = img.crop((left, upper, right, lower))
        if i == 0:
            first_img = cropped_img
        elif resize:
            cropped_img = resize_to_match_height(cropped_img, first_img.height)

        images.append(cropped_img)
        img.close()

    # Ensure all images have the same DPI
    dpis = [img.info.get("dpi", (72, 72)) for img in images]
    if not all(dpi == dpis[0] for dpi in dpis):
        print("Warning: Images have different DPI. Using DPI of the first image.")

    dpi = dpis[0]

    # Get max width and height of all images
    widths, heights = zip(*(img.size for img in images))

    max_width = max(widths)
    max_height = max(heights)
    print(max_width, max_height)

    # Calculate rows
    rows = math.ceil(len(images) / columns)

    # Create new blank image
    combined_width = columns * max_width
    combined_height = rows * max_height

    combined_image = Image.new("RGB",
                               (combined_width, combined_height),
                               color=(255, 255, 255))

    # Get font
    font = get_dynamic_font(dpi[0], height_in_inches=index_h)

    # Paste images into the combined image
    for index, image in enumerate(images):
        row = index // columns
        col = index % columns
        x = col * max_width
        y = row * max_height
        combined_image.paste(image, (x, y))
        draw = ImageDraw.Draw(combined_image)
        draw.text((x+10, y+10), labels[index], font=font, fill=(0, 0, 0))

    if resize:
        # Create a background image
        bg = Image.new('RGB', combined_image.size, background_color)
        
        # Get the difference between image and background
        diff = ImageChops.difference(combined_image, bg)
        
        # Get the bounding box of the non-background area
        bbox = diff.getbbox()
        if bbox is None:
            print("No figure content detected.")
            return img  # return original
        
        # Apply margin
        left = max(bbox[0] - margin, 0)
        upper = max(bbox[1] - margin, 0)
        right = min(bbox[2] + margin, combined_image.width)
        lower = min(bbox[3] + margin, combined_image.height)
        
        combined_image = combined_image.crop((left, upper, right, lower))
    
    # Save the result with DPI
    combined_image.save(output_path, dpi=dpi)
    print(f"Saved combined image to {output_path} with DPI {dpi}")

In [9]:
nCols = 2
indexH = 0.2
# figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/stats'
# figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/stats/vs_AERONET_L1.5'
# filetmp = '{figsdir}/{plotType}.{loopname}.{wvltag}.{timeRange}.png'
# figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/cycles/obs'
# filetmp = '{figsdir}/{obsname}/{plotType}.{loopname}.{wvltag}.{timeRange}.png'
# outfiletmp = '{savedir}/{plotType}.{set_type}.{wvltag}.{timeRange}.png'

figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/gridded'
filetmp = '{figsdir}/{plotType}.{loopname}.{timeRange}.png'
outfiletmp = '{savedir}/{plotType}.{set_type}.{timeRange}.png'
savedir = '/glade/work/swei/projects/mmm.pace_aod/plots/combined'
plotType = 'omb_mean'
set_dict = {
    'main': [
        'pace_aod',
        'modis_aqua_aod',
        # 'pace_aod-modis_aqua_aod',
        'viirs_aod_dt_n20',
        # 'pace_aod-viirs_aod_dt_n20',
        'viirs_aod_db_n20',
        # 'pace_aod-viirs_aod_db_n20',
        # 'merra2',
    ],
    'sup': [
        'pace_aod',
        'modis_terra_aod',
        'viirs_aod_dt_npp',
        'viirs_aod_db_npp',
    ]
}
set_type = 'main'
wvltag = '550nm'
timeRange = '202411'
# timeRange = '2024110100_2024113018'
loop_list = set_dict[set_type]
margin = 60

outfile = outfiletmp.format(savedir=savedir, plotType=plotType, set_type=set_type, wvltag=wvltag, timeRange=timeRange)

In [8]:
image_paths = []
for loopname in loop_list:
    tmpfile = filetmp.format(figsdir=figsdir, loopname=loopname, plotType=plotType, wvltag=wvltag, timeRange=timeRange)
    if not os.path.exists(tmpfile):
        raise Exception(f'{tmpfile} does not exist')
    image_paths.append(tmpfile)
combine_images(image_paths, outfile, columns=nCols, margin=margin, index_h=indexH)

Exception: /glade/work/swei/projects/mmm.pace_aod/plots/stats/all.hist2d.pace_aod.*.2024110100_2024113018.png does not exist

In [10]:
# Run with specified file list:
nCols = 3
indexH = 0.2
margin = 60
# figdir = '/glade/work/swei/Git/JEDI-METplus/output/wrfchem_evaluate/plots/2dmap/pandora_no2_total-wxaq'
figdir = '/glade/work/swei/projects/mmm.pace_aod/plots/stats'
savedir = '/glade/work/swei/projects/mmm.pace_aod/plots/combined'
# timeRange = '2024082419'
# timeRange = '2024082201_2024090100'
timeRange = '2024110100_2024113018'
outfile = f'{savedir}/all.hist2d.main.{timeRange}.png'
image_paths = [
    f"{figdir}/all.hist2d.pace_aod.550nm.2024110100_2024113018.png",
    f"{figdir}/all.hist2d.modis_aqua_aod.550nm.2024110100_2024113018.png",
    f"{figdir}/all.hist2d.viirs_aod_dt_n20.550nm.2024110100_2024113018.png",
    f"{figdir}/all.hist2d.viirs_aod_db_n20.550nm.2024110100_2024113018.png",
    f"{figdir}/all.hist2d.aeronet_l15_aod.500nm.2024110100_2024113018.png",
    # f"{figdir}/Boston.ObsValue.nitrogendioxideTotal.f19.2024082419.png",
    # f"{figdir}/Toronto.ObsValue.nitrogendioxideTotal.f19.2024082419.png",
    # f"{figdir}/NewYork.hofx.nitrogendioxideTotal.f19.2024082419.png",
    # f"{figdir}/Boston.hofx.nitrogendioxideTotal.f19.2024082419.png",
    # f"{figdir}/Toronto.hofx.nitrogendioxideTotal.f19.2024082419.png",
    # f"{figdir}/bias.TROPOMI_TEMPO.2024082201_2024090100.png",
    # f"{figdir}/rmse.TROPOMI_TEMPO.2024082201_2024090100.png",
    # f"{figdir}/tropomi_s5p_no2_troposphere-wxaq/ObsValue_nitrogendioxideColumn.f19.2024082419.png",
    # f"{figdir}/tropomi_s5p_no2_troposphere-wxaq/hofx_nitrogendioxideColumn.f19.2024082419.png",
    # f"{figdir}/tempo_no2_tropo-wxaq/ObsValue_nitrogendioxideColumn.f19.2024082419.png",
    # f"{figdir}/tempo_no2_tropo-wxaq/hofx_nitrogendioxideColumn.f19.2024082419.png",
]

combine_images(image_paths, outfile, columns=nCols, margin=margin, index_h=indexH, resize=True)

(3529, 3529)
(3529, 3529)
(3529, 3529)
(3529, 3529)
(3529, 3529)
3118 3352
Saved combined image to /glade/work/swei/projects/mmm.pace_aod/plots/combined/all.hist2d.main.2024110100_2024113018.png with DPI (599.9988, 599.9988)
